In [2]:
from torch.utils.data import Dataset, DataLoader
from torch import nn, from_numpy, optim
import numpy as np
#pandas- librărie pentru lucrul cu fișierele
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelBinarizer

In [3]:
!pwd
df=pd.read_csv("iris.csv",header=0, error_bad_lines=False, engine='python')

/home/ioana/faculta/si/regresie/my_proj


In [4]:
df.values

array([[1, 5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
       [2, 4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
       [3, 4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [4, 4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
       [5, 5.0, 3.6, 1.4, 0.2, 'Iris-setosa'],
       [6, 5.4, 3.9, 1.7, 0.4, 'Iris-setosa'],
       [7, 4.6, 3.4, 1.4, 0.3, 'Iris-setosa'],
       [8, 5.0, 3.4, 1.5, 0.2, 'Iris-setosa'],
       [9, 4.4, 2.9, 1.4, 0.2, 'Iris-setosa'],
       [10, 4.9, 3.1, 1.5, 0.1, 'Iris-setosa'],
       [11, 5.4, 3.7, 1.5, 0.2, 'Iris-setosa'],
       [12, 4.8, 3.4, 1.6, 0.2, 'Iris-setosa'],
       [13, 4.8, 3.0, 1.4, 0.1, 'Iris-setosa'],
       [14, 4.3, 3.0, 1.1, 0.1, 'Iris-setosa'],
       [15, 5.8, 4.0, 1.2, 0.2, 'Iris-setosa'],
       [16, 5.7, 4.4, 1.5, 0.4, 'Iris-setosa'],
       [17, 5.4, 3.9, 1.3, 0.4, 'Iris-setosa'],
       [18, 5.1, 3.5, 1.4, 0.3, 'Iris-setosa'],
       [19, 5.7, 3.8, 1.7, 0.3, 'Iris-setosa'],
       [20, 5.1, 3.8, 1.5, 0.3, 'Iris-setosa'],
       [21, 5.4, 3.4, 1.7, 0.2, 'Iris-setosa'],
 

Here I transform the dataset. The values are read from the csv. 
For the labels I used LabelBinarizer. The LabelBinarized acts as follows:
If I have 3 labels : "A", "B", "C" and I use the label binarizer each label is transformed into a vector that contains n elements (n = nr of distinct label) and all the values are 0 except the one that correspnds to the label.
Ex1:
l1 A -> [1, 0, 0]
l2 B -> [0, 1, 0]
l3 C -> [0, 0, 1]
I used this encoding because each label is a string. 

In [5]:
class DiabetesDataset(Dataset):
    """ Diabetes dataset."""
    # Initialize your data, download, etc.
    def __init__(self):
        #Citim setul de date
        df=pd.read_csv("iris.csv",header=0)
        xy = df.values
        self.len = xy.shape[0]
        #Vom folosi ca input toate valorile mai puțin ultima coloană
        self.x_data = torch.from_numpy(np.array(xy[:, 0:-1], dtype=np.float32))
        #Outputul este ultima coloana binarizata
        encoder = LabelBinarizer()
        transformed_label = encoder.fit_transform(xy[:, [-1]])
        self.y_data = torch.from_numpy(np.array(transformed_label, dtype=np.float32))
        

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [6]:
dataset = DiabetesDataset()
#DataLoader - un utilitar ce ne ajută să împărțim setul de date pe batch-uri și astfel să facem antrenare în mod Mini-Batch
train_loader = DataLoader(dataset=dataset,
                          batch_size=8,
                          shuffle=True,
                          num_workers=1)

In [7]:
dataset[0]

(tensor([1.0000, 5.1000, 3.5000, 1.4000, 0.2000]), tensor([1., 0., 0.]))

NN has 3 layers.
First layer has 5 inputs, associated with the 5 fetures read from the dataset.
The second layer is the hidden layer that has 6 neurons and the sigmoid function is used.
The 3rd layer has 4 neurons and relu is used.
The 4th layer has 3 neurons corresponding to the 3 different classes that I need. The labels are Strings encoded but the output has to be a probability between 0 and 1, for a flower beeing categorized as one type. The above encodation from the example Ex1 can be translated to :
l1 -> probability of 1 of the object being class A, and prob of 0 the obj being of class B or class C
l2 -> probability of 1 of the object being class B, and prob of 0 the obj being of class A or class C
l3 -> probability of 1 of the object being class C, and prob of 0 the obj being of class A or class B

In [22]:
class Model(nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l1 = nn.Linear(5, 6)
        self.l2 = nn.Linear(6, 4)
        self.l3 = nn.Linear(4, 3)

        self.sigmoid = nn.Sigmoid()
        #self.relu = nn.functional.relu(inplace=False)

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x))
        out2 = nn.functional.relu(input=self.l2(out1), inplace=False)
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred

In [29]:
model=Model()

I sed 3 criterions for the output because I have 3 different outputs, each showing the probability of am object being labeld as "A", "B" or "C" (actually each output neuron is corresponding to a complicated flower name like Iris-versicolor, but for keeping it simple I used A, B, C)

In [30]:
criterion1 = nn.BCELoss(reduction='sum')
criterion2 = nn.BCELoss(reduction='sum')
criterion3 = nn.BCELoss(reduction='sum')
optimizer = optim.SGD(model.parameters(), lr=0.0025)

The loss of the 3 outputs is then summed. It has to be divided by the batch size, in order to be the real loss.

In [31]:
# Training loop
for epoch in range(2000):
  for i, data in enumerate(train_loader, 0):
      # get the inputs
      inputs, labels = data
    
      # Forward pass: Compute predicted y by passing x to the model
      y_pred = model(inputs)
    
      #computing the loss for each output
      loss1 = criterion1(y_pred[:, 0], labels[:, 0])
      loss2 = criterion2(y_pred[:, 1], labels[:, 1])
      loss3 = criterion3(y_pred[:, 2], labels[:, 2])
      # Compute aThe total loss
      loss = (loss1 + loss2 + loss3) / 8.0
      if (epoch % 10 == 0):
          print(f'Epoch {epoch + 1} | Batch: {i+1} | Loss: {loss.item():.4f}')
            
      # Zero gradients, perform a backward pass, and update the weights.
      optimizer.zero_grad()
        
      loss.backward()
      optimizer.step()
print(y_pred)
print(labels)

Epoch 1 | Batch: 1 | Loss: 2.4209
Epoch 1 | Batch: 2 | Loss: 2.4959
Epoch 1 | Batch: 3 | Loss: 2.6628
Epoch 1 | Batch: 4 | Loss: 2.4772
Epoch 1 | Batch: 5 | Loss: 2.5659
Epoch 1 | Batch: 6 | Loss: 2.5054
Epoch 1 | Batch: 7 | Loss: 2.6484
Epoch 1 | Batch: 8 | Loss: 2.4724
Epoch 1 | Batch: 9 | Loss: 2.4969
Epoch 1 | Batch: 10 | Loss: 2.4226
Epoch 1 | Batch: 11 | Loss: 2.7993
Epoch 1 | Batch: 12 | Loss: 2.6251
Epoch 1 | Batch: 13 | Loss: 2.5301
Epoch 1 | Batch: 14 | Loss: 2.5259
Epoch 1 | Batch: 15 | Loss: 2.4702
Epoch 1 | Batch: 16 | Loss: 2.4242
Epoch 1 | Batch: 17 | Loss: 2.4598
Epoch 1 | Batch: 18 | Loss: 2.4440
Epoch 1 | Batch: 19 | Loss: 1.8824
Epoch 11 | Batch: 1 | Loss: 2.2230
Epoch 11 | Batch: 2 | Loss: 2.2526
Epoch 11 | Batch: 3 | Loss: 2.1993
Epoch 11 | Batch: 4 | Loss: 2.1964
Epoch 11 | Batch: 5 | Loss: 2.2020
Epoch 11 | Batch: 6 | Loss: 2.1939
Epoch 11 | Batch: 7 | Loss: 2.2754
Epoch 11 | Batch: 8 | Loss: 2.1897
Epoch 11 | Batch: 9 | Loss: 2.2066
Epoch 11 | Batch: 10 | Loss: 

Epoch 131 | Batch: 1 | Loss: 1.8940
Epoch 131 | Batch: 2 | Loss: 1.8888
Epoch 131 | Batch: 3 | Loss: 1.8809
Epoch 131 | Batch: 4 | Loss: 1.8501
Epoch 131 | Batch: 5 | Loss: 1.8691
Epoch 131 | Batch: 6 | Loss: 1.9101
Epoch 131 | Batch: 7 | Loss: 1.8716
Epoch 131 | Batch: 8 | Loss: 1.8611
Epoch 131 | Batch: 9 | Loss: 1.8616
Epoch 131 | Batch: 10 | Loss: 1.8578
Epoch 131 | Batch: 11 | Loss: 1.8800
Epoch 131 | Batch: 12 | Loss: 1.8456
Epoch 131 | Batch: 13 | Loss: 1.8643
Epoch 131 | Batch: 14 | Loss: 1.8618
Epoch 131 | Batch: 15 | Loss: 1.8969
Epoch 131 | Batch: 16 | Loss: 1.8979
Epoch 131 | Batch: 17 | Loss: 1.8500
Epoch 131 | Batch: 18 | Loss: 1.8693
Epoch 131 | Batch: 19 | Loss: 1.4073
Epoch 141 | Batch: 1 | Loss: 1.8160
Epoch 141 | Batch: 2 | Loss: 1.8550
Epoch 141 | Batch: 3 | Loss: 1.8754
Epoch 141 | Batch: 4 | Loss: 1.8679
Epoch 141 | Batch: 5 | Loss: 1.8849
Epoch 141 | Batch: 6 | Loss: 1.8716
Epoch 141 | Batch: 7 | Loss: 1.8567
Epoch 141 | Batch: 8 | Loss: 1.8625
Epoch 141 | Batch:

Epoch 251 | Batch: 1 | Loss: 1.5611
Epoch 251 | Batch: 2 | Loss: 1.4460
Epoch 251 | Batch: 3 | Loss: 1.4173
Epoch 251 | Batch: 4 | Loss: 1.3258
Epoch 251 | Batch: 5 | Loss: 1.6312
Epoch 251 | Batch: 6 | Loss: 1.4420
Epoch 251 | Batch: 7 | Loss: 1.8363
Epoch 251 | Batch: 8 | Loss: 1.4470
Epoch 251 | Batch: 9 | Loss: 1.7399
Epoch 251 | Batch: 10 | Loss: 1.4033
Epoch 251 | Batch: 11 | Loss: 1.3416
Epoch 251 | Batch: 12 | Loss: 1.4369
Epoch 251 | Batch: 13 | Loss: 1.2598
Epoch 251 | Batch: 14 | Loss: 1.5523
Epoch 251 | Batch: 15 | Loss: 1.3273
Epoch 251 | Batch: 16 | Loss: 1.5900
Epoch 251 | Batch: 17 | Loss: 1.5585
Epoch 251 | Batch: 18 | Loss: 1.4411
Epoch 251 | Batch: 19 | Loss: 1.0242
Epoch 261 | Batch: 1 | Loss: 1.4710
Epoch 261 | Batch: 2 | Loss: 1.3768
Epoch 261 | Batch: 3 | Loss: 1.2134
Epoch 261 | Batch: 4 | Loss: 1.3711
Epoch 261 | Batch: 5 | Loss: 1.4071
Epoch 261 | Batch: 6 | Loss: 1.5568
Epoch 261 | Batch: 7 | Loss: 1.3381
Epoch 261 | Batch: 8 | Loss: 1.4344
Epoch 261 | Batch:

Epoch 371 | Batch: 1 | Loss: 0.9667
Epoch 371 | Batch: 2 | Loss: 0.6808
Epoch 371 | Batch: 3 | Loss: 1.0206
Epoch 371 | Batch: 4 | Loss: 1.2515
Epoch 371 | Batch: 5 | Loss: 1.1328
Epoch 371 | Batch: 6 | Loss: 1.2005
Epoch 371 | Batch: 7 | Loss: 0.9002
Epoch 371 | Batch: 8 | Loss: 0.5412
Epoch 371 | Batch: 9 | Loss: 0.9023
Epoch 371 | Batch: 10 | Loss: 1.3829
Epoch 371 | Batch: 11 | Loss: 1.1571
Epoch 371 | Batch: 12 | Loss: 1.0511
Epoch 371 | Batch: 13 | Loss: 0.9978
Epoch 371 | Batch: 14 | Loss: 0.9588
Epoch 371 | Batch: 15 | Loss: 1.0567
Epoch 371 | Batch: 16 | Loss: 0.8788
Epoch 371 | Batch: 17 | Loss: 1.2494
Epoch 371 | Batch: 18 | Loss: 1.1121
Epoch 371 | Batch: 19 | Loss: 1.1240
Epoch 381 | Batch: 1 | Loss: 1.1859
Epoch 381 | Batch: 2 | Loss: 0.6984
Epoch 381 | Batch: 3 | Loss: 1.4967
Epoch 381 | Batch: 4 | Loss: 0.9140
Epoch 381 | Batch: 5 | Loss: 1.0077
Epoch 381 | Batch: 6 | Loss: 1.1378
Epoch 381 | Batch: 7 | Loss: 1.1225
Epoch 381 | Batch: 8 | Loss: 1.0263
Epoch 381 | Batch:

Epoch 491 | Batch: 1 | Loss: 1.1134
Epoch 491 | Batch: 2 | Loss: 0.8229
Epoch 491 | Batch: 3 | Loss: 1.2688
Epoch 491 | Batch: 4 | Loss: 1.0712
Epoch 491 | Batch: 5 | Loss: 0.8297
Epoch 491 | Batch: 6 | Loss: 0.8811
Epoch 491 | Batch: 7 | Loss: 1.0423
Epoch 491 | Batch: 8 | Loss: 1.1481
Epoch 491 | Batch: 9 | Loss: 0.4573
Epoch 491 | Batch: 10 | Loss: 0.9031
Epoch 491 | Batch: 11 | Loss: 0.8208
Epoch 491 | Batch: 12 | Loss: 0.7280
Epoch 491 | Batch: 13 | Loss: 0.6895
Epoch 491 | Batch: 14 | Loss: 0.7407
Epoch 491 | Batch: 15 | Loss: 0.8187
Epoch 491 | Batch: 16 | Loss: 0.9653
Epoch 491 | Batch: 17 | Loss: 1.1632
Epoch 491 | Batch: 18 | Loss: 0.9428
Epoch 491 | Batch: 19 | Loss: 0.7005
Epoch 501 | Batch: 1 | Loss: 0.9496
Epoch 501 | Batch: 2 | Loss: 1.4315
Epoch 501 | Batch: 3 | Loss: 0.7860
Epoch 501 | Batch: 4 | Loss: 0.8624
Epoch 501 | Batch: 5 | Loss: 0.9208
Epoch 501 | Batch: 6 | Loss: 0.9339
Epoch 501 | Batch: 7 | Loss: 0.8552
Epoch 501 | Batch: 8 | Loss: 0.5712
Epoch 501 | Batch:

...

Epoch 1451 | Batch: 1 | Loss: 0.6690
Epoch 1451 | Batch: 2 | Loss: 0.5547
Epoch 1451 | Batch: 3 | Loss: 0.4030
Epoch 1451 | Batch: 4 | Loss: 0.2994
Epoch 1451 | Batch: 5 | Loss: 0.9914
Epoch 1451 | Batch: 6 | Loss: 3.3051
Epoch 1451 | Batch: 7 | Loss: 0.2413
Epoch 1451 | Batch: 8 | Loss: 0.8132
Epoch 1451 | Batch: 9 | Loss: 0.3097
Epoch 1451 | Batch: 10 | Loss: 0.9180
Epoch 1451 | Batch: 11 | Loss: 1.1705
Epoch 1451 | Batch: 12 | Loss: 0.8551
Epoch 1451 | Batch: 13 | Loss: 0.2201
Epoch 1451 | Batch: 14 | Loss: 0.5345
Epoch 1451 | Batch: 15 | Loss: 0.3304
Epoch 1451 | Batch: 16 | Loss: 0.7010
Epoch 1451 | Batch: 17 | Loss: 0.5892
Epoch 1451 | Batch: 18 | Loss: 0.4757
Epoch 1451 | Batch: 19 | Loss: 0.2436
Epoch 1461 | Batch: 1 | Loss: 1.5420
Epoch 1461 | Batch: 2 | Loss: 0.1494
Epoch 1461 | Batch: 3 | Loss: 0.8861
Epoch 1461 | Batch: 4 | Loss: 0.5013
Epoch 1461 | Batch: 5 | Loss: 0.7934
Epoch 1461 | Batch: 6 | Loss: 0.6286
Epoch 1461 | Batch: 7 | Loss: 0.6146
Epoch 1461 | Batch: 8 | Loss

Epoch 1571 | Batch: 1 | Loss: 0.2564
Epoch 1571 | Batch: 2 | Loss: 0.2233
Epoch 1571 | Batch: 3 | Loss: 0.1745
Epoch 1571 | Batch: 4 | Loss: 0.6708
Epoch 1571 | Batch: 5 | Loss: 0.8981
Epoch 1571 | Batch: 6 | Loss: 1.2935
Epoch 1571 | Batch: 7 | Loss: 0.1437
Epoch 1571 | Batch: 8 | Loss: 0.1669
Epoch 1571 | Batch: 9 | Loss: 0.8341
Epoch 1571 | Batch: 10 | Loss: 0.9379
Epoch 1571 | Batch: 11 | Loss: 0.4653
Epoch 1571 | Batch: 12 | Loss: 0.3883
Epoch 1571 | Batch: 13 | Loss: 0.4476
Epoch 1571 | Batch: 14 | Loss: 0.4561
Epoch 1571 | Batch: 15 | Loss: 0.8973
Epoch 1571 | Batch: 16 | Loss: 0.4153
Epoch 1571 | Batch: 17 | Loss: 0.4841
Epoch 1571 | Batch: 18 | Loss: 0.2249
Epoch 1571 | Batch: 19 | Loss: 0.6246
Epoch 1581 | Batch: 1 | Loss: 0.4740
Epoch 1581 | Batch: 2 | Loss: 0.1883
Epoch 1581 | Batch: 3 | Loss: 0.8751
Epoch 1581 | Batch: 4 | Loss: 0.2151
Epoch 1581 | Batch: 5 | Loss: 0.9076
Epoch 1581 | Batch: 6 | Loss: 1.4559
Epoch 1581 | Batch: 7 | Loss: 0.3761
Epoch 1581 | Batch: 8 | Loss

Epoch 1691 | Batch: 1 | Loss: 0.6874
Epoch 1691 | Batch: 2 | Loss: 0.6173
Epoch 1691 | Batch: 3 | Loss: 0.5269
Epoch 1691 | Batch: 4 | Loss: 0.8998
Epoch 1691 | Batch: 5 | Loss: 0.5986
Epoch 1691 | Batch: 6 | Loss: 0.6525
Epoch 1691 | Batch: 7 | Loss: 1.2656
Epoch 1691 | Batch: 8 | Loss: 1.4893
Epoch 1691 | Batch: 9 | Loss: 0.9058
Epoch 1691 | Batch: 10 | Loss: 0.7815
Epoch 1691 | Batch: 11 | Loss: 0.5756
Epoch 1691 | Batch: 12 | Loss: 1.1149
Epoch 1691 | Batch: 13 | Loss: 0.4967
Epoch 1691 | Batch: 14 | Loss: 1.1283
Epoch 1691 | Batch: 15 | Loss: 1.5166
Epoch 1691 | Batch: 16 | Loss: 1.4949
Epoch 1691 | Batch: 17 | Loss: 1.5831
Epoch 1691 | Batch: 18 | Loss: 1.0105
Epoch 1691 | Batch: 19 | Loss: 0.8988
Epoch 1701 | Batch: 1 | Loss: 1.7089
Epoch 1701 | Batch: 2 | Loss: 0.6081
Epoch 1701 | Batch: 3 | Loss: 0.9235
Epoch 1701 | Batch: 4 | Loss: 0.8568
Epoch 1701 | Batch: 5 | Loss: 0.7309
Epoch 1701 | Batch: 6 | Loss: 0.6350
Epoch 1701 | Batch: 7 | Loss: 1.0795
Epoch 1701 | Batch: 8 | Loss

Epoch 1811 | Batch: 1 | Loss: 0.3917
Epoch 1811 | Batch: 2 | Loss: 0.7253
Epoch 1811 | Batch: 3 | Loss: 0.3065
Epoch 1811 | Batch: 4 | Loss: 0.5919
Epoch 1811 | Batch: 5 | Loss: 0.6843
Epoch 1811 | Batch: 6 | Loss: 0.2774
Epoch 1811 | Batch: 7 | Loss: 0.2109
Epoch 1811 | Batch: 8 | Loss: 0.2870
Epoch 1811 | Batch: 9 | Loss: 0.9332
Epoch 1811 | Batch: 10 | Loss: 0.3879
Epoch 1811 | Batch: 11 | Loss: 0.9339
Epoch 1811 | Batch: 12 | Loss: 1.0676
Epoch 1811 | Batch: 13 | Loss: 1.0088
Epoch 1811 | Batch: 14 | Loss: 1.3260
Epoch 1811 | Batch: 15 | Loss: 0.3598
Epoch 1811 | Batch: 16 | Loss: 0.6014
Epoch 1811 | Batch: 17 | Loss: 0.6077
Epoch 1811 | Batch: 18 | Loss: 0.6059
Epoch 1811 | Batch: 19 | Loss: 0.2046
Epoch 1821 | Batch: 1 | Loss: 0.5504
Epoch 1821 | Batch: 2 | Loss: 0.2941
Epoch 1821 | Batch: 3 | Loss: 0.2872
Epoch 1821 | Batch: 4 | Loss: 0.3745
Epoch 1821 | Batch: 5 | Loss: 0.6707
Epoch 1821 | Batch: 6 | Loss: 0.1616
Epoch 1821 | Batch: 7 | Loss: 0.4642
Epoch 1821 | Batch: 8 | Loss

Epoch 1931 | Batch: 1 | Loss: 0.2334
Epoch 1931 | Batch: 2 | Loss: 0.7837
Epoch 1931 | Batch: 3 | Loss: 0.3350
Epoch 1931 | Batch: 4 | Loss: 0.2188
Epoch 1931 | Batch: 5 | Loss: 1.4833
Epoch 1931 | Batch: 6 | Loss: 0.4313
Epoch 1931 | Batch: 7 | Loss: 0.5022
Epoch 1931 | Batch: 8 | Loss: 0.3051
Epoch 1931 | Batch: 9 | Loss: 0.4614
Epoch 1931 | Batch: 10 | Loss: 0.3979
Epoch 1931 | Batch: 11 | Loss: 0.3106
Epoch 1931 | Batch: 12 | Loss: 0.3887
Epoch 1931 | Batch: 13 | Loss: 0.6333
Epoch 1931 | Batch: 14 | Loss: 0.2531
Epoch 1931 | Batch: 15 | Loss: 0.2378
Epoch 1931 | Batch: 16 | Loss: 0.6056
Epoch 1931 | Batch: 17 | Loss: 0.1293
Epoch 1931 | Batch: 18 | Loss: 1.2215
Epoch 1931 | Batch: 19 | Loss: 0.4454
Epoch 1941 | Batch: 1 | Loss: 0.1853
Epoch 1941 | Batch: 2 | Loss: 0.4389
Epoch 1941 | Batch: 3 | Loss: 0.6199
Epoch 1941 | Batch: 4 | Loss: 0.3054
Epoch 1941 | Batch: 5 | Loss: 1.0563
Epoch 1941 | Batch: 6 | Loss: 0.5946
Epoch 1941 | Batch: 7 | Loss: 0.1909
Epoch 1941 | Batch: 8 | Loss

The loss for the final iteration through the data is 
Epoch 1991 | Batch: 1 | Loss: 0.8679
Epoch 1991 | Batch: 2 | Loss: 0.8723
Epoch 1991 | Batch: 3 | Loss: 0.5212
Epoch 1991 | Batch: 4 | Loss: 1.2189
Epoch 1991 | Batch: 5 | Loss: 0.8763
Epoch 1991 | Batch: 6 | Loss: 0.8674
Epoch 1991 | Batch: 7 | Loss: 1.2255
Epoch 1991 | Batch: 8 | Loss: 0.6967
Epoch 1991 | Batch: 9 | Loss: 0.8668
Epoch 1991 | Batch: 10 | Loss: 0.8690
Epoch 1991 | Batch: 11 | Loss: 1.0424
Epoch 1991 | Batch: 12 | Loss: 1.2174
Epoch 1991 | Batch: 13 | Loss: 1.0434
Epoch 1991 | Batch: 14 | Loss: 0.6964
Epoch 1991 | Batch: 15 | Loss: 1.0453
Epoch 1991 | Batch: 16 | Loss: 0.8700
Epoch 1991 | Batch: 17 | Loss: 1.0399
Epoch 1991 | Batch: 18 | Loss: 0.8724
Epoch 1991 | Batch: 19 | Loss: 0.6933

The loss has a good value because the algorithm converges.